In [ ]:
import huggingface_hub

huggingface_hub.login()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\KDP-25\.cache\huggingface\token
Login successful


In [ ]:
from transformers import (
    LlamaForCausalLM,
    AutoTokenizer,
    AutoModelForCausalLM,
)

model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)


#### 토큰 분류 모델 설계 과정

*   데이터 전처리
*   학습 파라미터 설정
*   학습 및 평가


1. 데이터 전처리<hr>


In [4]:
# 데이터 불러오기
import pandas as pd
import json, os


In [ ]:
# 테스트 데이터 불러오기
test_list=os.listdir('./character_test/')
for test in test_list:
    if test_list.index(test)==0:
        testingDF=pd.read_json(f'./character_test/{test}', encoding='utf-8')
    else:
        testingDF2=pd.read_json(f'./character_test/{test}', encoding='utf-8')
        testingDF=pd.concat([testingDF, testingDF2])
testingDF.to_csv('./testDF.csv', index=False)

#새 통합 데이터 부르기
tagDF=pd.read_json('./data/raw_data/learning_sentence.json')
text=pd.read_table('./data/raw_data/learning_sentence.txt')
tagDF['sentence']=text['text']

In [8]:
tagDF.head()

,tokens,ner_tags,sentence
0,"[오, 프, 커, 피, 에, 서, 아, 메, 리, 카, 노, 핫, 으, 로, 2, ...","[1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 4, 0, 0, 0, 5, ...",오프커피에서아메리카노핫으로2잔주문해주세요
1,"[롤, 키, 에, 서, 치, 즈, 롤, 케, 이, 크, 한, 조, 각, 이, 랑, ...","[1, 2, 0, 0, 3, 4, 4, 4, 4, 4, 5, 6, 6, 0, 0, ...",롤키에서치즈롤케이크한조각이랑아메리카노아이스로하나씩부탁드립니다
2,"[리, 스, 아, 라, 비, 카, 에, 서, 라, 떼, 에, 디, 카, 페, 인, ...","[1, 2, 2, 2, 2, 2, 0, 0, 3, 4, 0, 0, 0, 0, 0, ...","리스아라비카에서라떼에디카페인옵션으로두잔,라지사이즈로부탁드릴게요"
3,"[C, O, F, F, E, E, R, O, M, A, N, 에, 서, 플, 랫, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 3, 4, ...","COFFEEROMAN에서플랫화이트두잔,소이라떼하나로총3잔주문할게요"
4,"[고, 더, 커, 피, 에, 서, 카, 푸, 치, 노, 2, 잔, ,, 시, 나, ...","[1, 2, 2, 2, 0, 0, 3, 4, 4, 4, 5, 6, 0, 0, 0, ...","고더커피에서카푸치노2잔,시나몬추가로부탁드립니다"


In [9]:
# NER_TAG 맵핑
# - 매장명(STR), 메뉴(N), 수량(CNT)
id2label = {1: "B-STR", 2: "I-STR", 3: "B-N", 4: "I-N", 5: "B-CNT", 6: "I-CNT", 0: "O"}
label2id = {"B-STR": 1, "I-STR": 2, "B-N": 3, "I-N": 4, "B-CNT": 5, "I-CNT": 6, "O": 0}

# 맵핑 적용
tagDF["ner_tags"] = tagDF["ner_tags"].apply(
    lambda tags: [id2label[int(tag)] for tag in tags]
)


In [ ]:
tagDF.info(), tagDF.head(3)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812 entries, 0 to 811
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tokens    812 non-null    object
 1   ner_tags  812 non-null    object
 2   sentence  812 non-null    object
dtypes: object(3)
memory usage: 19.2+ KB


(None,
                                               tokens  \
 0  [오, 프, 커, 피, 에, 서, 아, 메, 리, 카, 노, 핫, 으, 로, 2, ...   
 1  [롤, 키, 에, 서, 치, 즈, 롤, 케, 이, 크, 한, 조, 각, 이, 랑, ...   
 2  [리, 스, 아, 라, 비, 카, 에, 서, 라, 떼, 에, 디, 카, 페, 인, ...   
 
                                             ner_tags  \
 0  [B-STR, I-STR, I-STR, I-STR, O, O, B-N, I-N, I...   
 1  [B-STR, I-STR, O, O, B-N, I-N, I-N, I-N, I-N, ...   
 2  [B-STR, I-STR, I-STR, I-STR, I-STR, I-STR, O, ...   
 
                              sentence  
 0              오프커피에서아메리카노핫으로2잔주문해주세요  
 1   롤키에서치즈롤케이크한조각이랑아메리카노아이스로하나씩부탁드립니다  
 2  리스아라비카에서라떼에디카페인옵션으로두잔,라지사이즈로부탁드릴게요  )

In [12]:
# Datasets  형태로 전환
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# 학습용, 검증용, 테스트용 데이터 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    tagDF.drop(["ner_tags"], axis=1), tagDF["ner_tags"], test_size=0.2, random_state=42
)
test_df=pd.read_csv('./testDF.csv')
x_test, y_test= testingDF['tokens'], testingDF['ner_tags']
print(x_train.shape, y_train.shape, x_valid.shape, y_valid.shape)


(649, 2) (649,) (163, 2) (163,)


In [18]:
type(testingDF['tokens'])

pandas.core.series.Series

In [20]:
# 허깅페이스 형태로 맞추기

train_data = {
    "sentence": x_train['sentence'],
    "tokens": x_train["tokens"],
    "ner_tags": y_train,
}
trainDS = Dataset.from_dict(train_data)
valid_data = {
    "sentence": x_valid['sentence'],
    "tokens": x_valid["tokens"],
    "ner_tags": y_valid,
}
validDS = Dataset.from_dict(valid_data)
test_data= {
    'tokens': x_test,
    'ner_tags': y_test
}
testDF= pd.read_csv('./test_data.csv')
testDS= Dataset.from_pandas(testDF)
dataset = DatasetDict({"train": trainDS, "valid": validDS,
                       'test': testDS})


In [ ]:
dataset
# 데이터셋의 형태
# dataset (dict 형태)
# - train  (list 형태-> 안의 요소가 dict로 구성)
# - sentence
# - toekns
# - labels

# - test    (list 형태)
# - sentence
# - toekns
# - labels


DatasetDict({
    train: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 649
    })
    valid: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 163
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 79
    })
})

In [ ]:
label_list = list(id2label.values())


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], padding=True, padding_side='right', is_split_into_words=True
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                try:
                    if label[word_idx] == "O":
                        label_ids.append(0)
                    else:
                        label_ids.append(int(label2id[label[word_idx]]))
                except ValueError:
                    label_ids.append(-100)
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_wnut = dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


2. 학습 파라미터 지정<hr>


In [ ]:
import evaluate
import numpy as np

# seqeval -> 분류에 대한 여러 평가지표 산출
seqeval = evaluate.load("seqeval")

labels = list(id2label.values())


def compute_metrics(p):
    predictions, labels = p

    # ['O' 레이블 가중치]*0.5 -> 'O' 레이블로만 예측되는 문제 개선(클래스 불균형 문제)
    for i in range(len(predictions)):
        for j in range(len(predictions[i])):
            predictions[i][j][0] = predictions[i][j][0] * 0.5

    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [ ]:
# 모델 설정
from transformers import (AutoModelForTokenClassification, 
                          LlamaForTokenClassification, TrainingArguments, 
                          Trainer, EarlyStoppingCallback
)


model = LlamaForTokenClassification.from_pretrained(
    model_name, num_labels=7, id2label=id2label, label2id=label2id
)
# , num_labels=7, id2label=id2label, label2id=label2id
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# 하이퍼 파라미터 설정
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=3)
training_args = TrainingArguments(
    output_dir="./llama_result_1",
    learning_rate=1e-4,
    # gradient_accumulation_steps=2,
    fp16=True,
    save_total_limit=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    dataloader_pin_memory=True,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
)
trainer.train()


  0%|          | 0/200 [00:00<?, ?it/s]c:\Users\KDT\anaconda3\envs\llama_38\lib\site-packages\transformers\models\llama\modeling_llama.py:655: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
                                                
 10%|█         | 20/200 [00:08<00:33,  5.30it/s]

{'eval_loss': 0.7833276391029358, 'eval_precision': 0.6720080120180271, 'eval_recall': 0.7530864197530864, 'eval_f1': 0.7102408044456204, 'eval_accuracy': 0.7107843137254902, 'eval_runtime': 0.5155, 'eval_samples_per_second': 306.489, 'eval_steps_per_second': 9.699, 'epoch': 1.0}


                                                   
 20%|██        | 40/200 [06:29<00:49,  3.24it/s]

{'eval_loss': 0.269212543964386, 'eval_precision': 0.9416642836717186, 'eval_recall': 0.9239618406285073, 'eval_f1': 0.9327290752018127, 'eval_accuracy': 0.9392156862745098, 'eval_runtime': 0.4383, 'eval_samples_per_second': 360.509, 'eval_steps_per_second': 11.409, 'epoch': 2.0}


                                                   
 30%|███       | 60/200 [12:45<00:40,  3.42it/s]

...

                                                    
 80%|████████  | 160/200 [44:28<00:12,  3.26it/s]

{'eval_loss': 0.1961502581834793, 'eval_precision': 0.9616577389343107, 'eval_recall': 0.9570707070707071, 'eval_f1': 0.9593587399803122, 'eval_accuracy': 0.9634803921568628, 'eval_runtime': 0.3882, 'eval_samples_per_second': 406.96, 'eval_steps_per_second': 12.878, 'epoch': 8.0}


 80%|████████  | 160/200 [50:46<12:41, 19.04s/it]

{'train_runtime': 3046.2766, 'train_samples_per_second': 2.075, 'train_steps_per_second': 0.066, 'train_loss': 0.5897091865539551, 'epoch': 8.0}


TrainOutput(global_step=160, training_loss=0.5897091865539551, metrics={'train_runtime': 3046.2766, 'train_samples_per_second': 2.075, 'train_steps_per_second': 0.066, 'total_flos': 1224173570120880.0, 'train_loss': 0.5897091865539551, 'epoch': 8.0})

In [ ]:
#테스트 데이터 평가
trainer_test = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback],
)
trainer.evaluate()


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from sklearn.metrics import *
import pandas as pd
import os, json


DEVICE='cuda'
test_dir= os.listdir('./character_test')
for test_file in test_dir:
    if test_dir.index(test_file)==0:
        testDF=pd.read_json(f'./character_test/{test_file}', encoding='utf-8')
    else:
        testDF2=pd.read_json(f'./character_test/{test_file}', encoding='utf-8')
        testDF=pd.concat([testDF, testDF2])
tokens = testDF['tokens']
ner_tags = testDF['ner_tags']



total_pred_list = []
total_target_list = []
for token, ner_tag in zip(tokens, ner_tags):
    tokenized_inputs = tokenizer(token, padding=True, padding_side='right' , is_split_into_words=True, return_tensors='pt')

    with torch.no_grad():
        output = model(
            input_ids = tokenized_inputs['input_ids'].to(DEVICE),
            attention_mask = tokenized_inputs['attention_mask'].to(DEVICE)
        )
    pred=output.logits
    pred_list = pred.argmax(axis=2).tolist()
    total_pred_list.extend(pred_list[0][:len(token)])
    total_target_list.extend([int(a) for a in ner_tag])

try:
    accuracy = accuracy_score(total_target_list, total_pred_list)
    f1 = f1_score(total_target_list, total_pred_list, average='weighted')
    mse = mean_squared_error(total_target_list, total_pred_list)
except Exception as e:
    print(e)
print(f"accuracy : {accuracy}")
print(f"f1_score : {f1}")
print(f"mse : {mse}")
print(classification_report(total_target_list, total_pred_list))

accuracy : 0.40524781341107874
f1_score : 0.3914151782498473
mse : 5.41649312786339
              precision    recall  f1-score   support

           0       0.42      0.29      0.34       585
           1       0.00      0.00      0.00        82
           2       0.58      0.72      0.64       389
           3       0.02      0.02      0.02       165
           4       0.50      0.57      0.53       854
           5       0.08      0.08      0.08       152
           6       0.11      0.10      0.10       174

    accuracy                           0.41      2401
   macro avg       0.24      0.25      0.25      2401
weighted avg       0.39      0.41      0.39      2401



In [ ]:
model.save_pretrained("./result_2")
tokenizer.save_pretrained("./result_2")


('./result_2\\tokenizer_config.json',
 './result_2\\special_tokens_map.json',
 './result_2\\tokenizer.json')